# Low-rank approximation on $\mathcal{P}(d)$ - the space of $d$-dimensional SPD matrices

In this notebook we want to get some intuition in different approaches for computing low-rank approximations for manifold-valued signals

In [92]:
using Manifolds
using Manopt
using LinearAlgebra
using Distributions

In [93]:
# initialize SPD power manifold P(d)^d1
d = 3  # size of the SPD matrices, i.e., ∈ R^d×d
M_base = SymmetricPositiveDefinite(d)
d1 = 50  # size of the signal
M = PowerManifold(M_base, NestedPowerRepresentation(), d1)

PowerManifold(SymmetricPositiveDefinite(3), NestedPowerRepresentation(), 50)

### Generate a data set

In [94]:
e = 1. * Matrix(I, d, d)
# compute basis on M_base
Θ = get_basis(M_base, e, DefaultOrthonormalBasis())
#  construct data
Q = vcat(fill(e, d1))
log_Q_data = zero_vector(M, Q)
# draw random tvectors 
epsilon = 1.
Xₑⁱ = zeros(manifold_dimension(M_base))
Xₑⁱ[1] = 1.
Xₑ = get_vector(M_base, e, Xₑⁱ, Θ)
for i in 1:d1
    # compute t(X + sY) ∈ T_e P(3)
    Yₑⁱ = randn(manifold_dimension(M_base))
    Yₑ = get_vector(M_base, e, Yₑⁱ/norm(Yₑⁱ), Θ) 
    t = randn(1)[1]
    s = epsilon * randn(1)[1]
    log_Q_data[i] = t * (Xₑ + s * Yₑ) # log_Q data ∈ T_e P(3)^d1
end
data = exp(M, Q, log_Q_data)  # ∈ P(3)^d1

50-element Vector{Matrix{Float64}}:
 [0.6406720296046633 0.3195057800708988 -0.017588408125458817; 0.3195057800708988 1.3894229748002302 0.040554454768382035; -0.01758840812545882 0.040554454768382014 0.5384181613378851]
 [1.0693065057340807 0.5094614127391066 -0.03181612400072143; 0.5094614127391066 1.440029703878923 0.010036638644243985; -0.0318161240007214 0.01003663864424397 0.8505625249859999]
 [9.463985194293352 0.06342647945207336 2.8188843258279626; 0.06342647945207347 1.942322955728859 1.3802637175127648; 2.8188843258279626 1.3802637175127652 3.199812819386082]
 [1.288753216944623 0.04703782445263574 -0.08880115092270296; 0.04703782445263574 1.013317817757992 -0.03734272459734039; -0.08880115092270296 -0.03734272459734039 0.9588478157123459]
 [1.9394612425441833 0.36324067423462375 -0.01712606820948212; 0.36324067423462375 1.1579575739504753 -0.19288698472165616; -0.01712606820948212 -0.19288698472165616 0.9661565333382042]
 [0.5656564273589889 -0.5522207547649167 0.8239095035

We want to investigate how reliable both ranks are, i.e., we don't want that something looks low-rank, but in reality is higher rank

In [95]:
# compute SVD
# compute Gramm Matrix
Gramm_Q = zeros(d1,d1)
for i in 1:d1
    for j in 1:d1
        Gramm_Q[i,j] = inner(M_base, e, log_Q_data[i], log_Q_data[j])
    end
end
# compute Σ and V
(evals, evecs) = eigen(Symmetric(Gramm_Q), d1-min(d1-1,manifold_dimension(M_base)-1):d1)
# compute low rank approximation of tangent vector
# expoentiate back
# compute error

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
6-element Vector{Float64}:
  14.343166619121657
  19.080682268023814
  51.71729927476629
  65.74411552134868
 141.79635872909628
 224.06272681823896
vectors:
50×6 Matrix{Float64}:
 -0.0929341   -0.000289204  -0.0854631    …  -0.0485924   -0.00641222
 -0.0241571    0.0468417    -0.0472569       -0.0320542   -0.0281325
  0.413571    -0.191431      0.0357102        0.0892892   -0.0915961
 -0.00668319   0.0200596     0.00106312       0.0144317   -0.014152
  0.0287814    0.0865582    -0.0289116        0.029358    -0.0342315
  0.277964    -0.28505      -0.686663     …  -0.227189     0.387461
  0.153508     0.157465     -0.0334352       -0.0184887    0.147012
  0.0250964    0.0274372     0.000338852      0.0185551    0.0128155
 -0.00893762   0.0242739     0.0185603       -0.00921028   0.0034027
 -0.296813    -0.402131      0.111449        -0.0796835   -0.0375156
  ⋮                                       ⋱                ⋮
 -0.1

In [107]:
# compute SVD
# B = get_basis(M_base, Q[1], DiagonalizingOrthonormalBasis(log(M_base,Q[1],data[1])))
B = get_basis(M_base, Q[1], DiagonalizingOrthonormalBasis(log_Q_data[1]))
# B = get_basis(M, Q, DiagonalizingOrthonormalBasis(log_Q_data)) # doesn't compute for powermanifolds -> we should better do it element-wise
B.data.eigenvalues
# compute rescaled Gramm Matrix
# compute Σ and V
# compute low rank approximation of tangent vector
# expoentiate back
# compute error

6-element Vector{Float64}:
 -0.0
 -0.003508740243259196
 -0.3061515379314268
 -0.0
 -0.24411010741439912
 -0.0